In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os

import pandas as pd
import json
import random

from subword_prompt_templates.MultiChoicePrompts import MultiChoicePrompts
from subword_prompt_templates.ClassificationPrompts import ClassificationPrompts

from utils import helpers

from openai import OpenAI

from matplotlib import pyplot as plt
from matplotlib.ticker import FuncFormatter

In [ ]:
models = ['MISTRALAI/MIXTRAL-8X7B-INSTRUCT-V0.1', 'allenai/OLMo-7B-Instruct', 'meta-llama/Llama-3-8b-chat-hf']
prompts = ['zero_shot', 'one_shot', 'few_shot', 'CoT-one', 'CoT-few', 'decomposite']
dataset = pd.read_csv(r"datasets\dataset_for_evaluation.csv", index_col=0)

TOGETHER_API_KEY = os.environ.get("TOGETHER_API_KEY")
client = OpenAI(api_key=TOGETHER_API_KEY, base_url='https://api.together.xyz/v1')

random.seed(42)

In [ ]:
results = []
mcp = MultiChoicePrompts()
ynp = ClassificationPrompts()
for model in models:
    for prompt_type in prompts:
        if '-' in prompt_type: # for CoT
            prompt_technique, shot = prompt_type.split('-')
            kwargs = {'shot': shot}
        else:
            prompt_technique = prompt_type
            kwargs = {}
        for row in dataset[:12].itertuples():
            wrong_choices = json.loads(row.Mulitple_Options.replace(" ", ", ").replace("'", '"'))
            correct_choice = row.Word
            choices = wrong_choices + [correct_choice]
            random.shuffle(choices)
            correct_choice_letter = chr(choices.index(correct_choice) + 65)
            multi_question = mcp.generate(prompt_technique, row.Category, *choices, **kwargs)
            yes_question = ynp.generate(prompt_technique, row.Category, row.Word, **kwargs)
            no_question = ynp.generate(prompt_technique, row.Category, random.choice(wrong_choices), **kwargs)
            multi_answer = helpers.generate_llm_response(client, multi_question, model)
            yes_answer = helpers.generate_llm_response(client, yes_question, model)
            no_answer = helpers.generate_llm_response(client, no_question, model)
            multi_match = helpers.match_answer(multi_answer)
            yes_match = helpers.match_answer(yes_answer)
            no_match = helpers.match_answer(no_answer)
            multi_eval = helpers.evaluate_response(multi_answer, correct_choice_letter)
            yes_eval = helpers.evaluate_response(yes_answer, "Yes")
            no_eval = helpers.evaluate_response(no_answer, "No")
            results.append((model, prompt_type, 'multi', multi_question, multi_answer, multi_match, correct_choice_letter, multi_eval))
            results.append((model, prompt_type, 'classification', yes_question, yes_answer, yes_match, "yes", yes_eval))
            results.append((model, prompt_type, 'classification', no_question, no_answer, no_match, "No", no_eval))
results_df = pd.DataFrame(results, columns=["Model", "Prompt Type", "Question type", "Question", "LLM Response", "LLM match", "Correct answer", "Eval"])
results_df.to_csv("check_evaluate.csv")
results_df.head()

# multi-choice results

In [ ]:
multi_pv = results_df[results_df['Question type']=='multi'].pivot_table(index='Model', columns='Prompt Type', values='Eval', aggfunc='mean')[prompts]
display(multi_pv)
helpers.plot_results(multi_pv, radnom_line=0.25) # for now the lines are not really correct

# Yes\No results

In [ ]:
classification_pv = results_df[results_df['Question type']=='classification'].pivot_table(index='Model', columns='Prompt Type', values='Eval', aggfunc='mean')[prompts]
display(classification_pv)
helpers.plot_results(classification_pv, radnom_line=0.5) # for now the lines are not really correct

# tests

In [ ]:
import re

gold_response = "A"

pattern = rf'\W*({re.escape(gold_response)})\W*\b'

full_pattern = rf'{pattern}|The correct .* (is|are){pattern}'

# Example strings to test
test_strings = ["A.", "A)", "[A]", "A", "An", "A+", "A*", "A ", " A ", "The A", "A is the correct answer",
                "AThe correct answer is (A)", "The correct letter is A)",
                '''The correct letter and word are:
[A] incredible
''',":\n[A", 'The correct letter and word are A. million']

# Check each string
for string in test_strings:
    match = re.match(full_pattern, string)
    if match:
        chosen_letter = [m for m in match.groups() if m is not None][-1]
        print(f"'{string}' starts with '{gold_response}'", f' - {chosen_letter}')
    else:
        print(f"'{string}' does not match the pattern.")

In [ ]:
import re

gold_response = "Yes"

pattern = rf'\W*{gold_response}\b'

# Example strings to test
test_strings = ["Yes.", "Yes)", "[Yes]", "Yes", "Yess", "Yes ", " Yes ", "The Yes", "Yes is the correct answer"]

# Check each string
for string in test_strings:
    if re.match(pattern, string):
        print(f"'{string}' starts with '{gold_response}'")
    else:
        print(f"'{string}' does not match the pattern.")
        